In [225]:
using CSV, DataFrames, GLM, Statistics, Dates, Gadfly, Random, MLBase, DecisionTree;
include("utils/precipitation.jl");
include("utils/random-forest.jl");
include("utils/reg-log.jl");

In [226]:
function partitionTrainTest(data, at = 0.8) # https://discourse.julialang.org/t/simple-tool-for-train-test-split/473/2
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    return data[train_idx,:], data[test_idx,:]
end

partitionTrainTest (generic function with 2 methods)

In [227]:
val_form = @formula(SURVERSE ~ McTavish_sum + McTavish_max3 + 
                               Bellevue_sum + Bellevue_max3 + 
                               Assomption_sum + Assomption_max3 + 
                               Trudeau_sum + Trudeau_max3 + 
                               StHubert_sum + StHubert_max3);

In [228]:
names_ft = [:McTavish_sum, :McTavish_max, :McTavish_max3,
            :Bellevue_sum, :Bellevue_max, :Bellevue_max3,
            :Assomption_sum, :Assomption_max, :Assomption_max3,
            :Trudeau_sum, :Trudeau_max, :Trudeau_max3,
            :StHubert_sum, :StHubert_max, :StHubert_max3];

In [229]:
params_rf = DataFrame(param=String[], min=Int8[], max=Int8[], step=Int8[]);

push!(params_rf, ["nft", 3, 7, 1]);
push!(params_rf, ["ntrees", 75, 120, 15]);
push!(params_rf, ["podata", 75, 95, 5]);
push!(params_rf, ["maxd", 5, 12, 1]);

params_rf

,param,min,max,step
,String,Int8,Int8,Int8
1,nft,3,7,1
2,ntrees,75,120,15
3,podata,75,95,5
4,maxd,5,12,1


## Ouvrage 3260

In [221]:
data_set = CSV.read("data/parsed/ouvrage_3260.csv");
train_set, val_set = partitionTrainTest(data_set);
val_labels = val_set[!, :SURVERSE];
size(data_set)

(1097, 17)

#### Train GLM

In [222]:
val_model = glm(val_form, train_set, Bernoulli(), LogitLink())
val_pred_glm = GLM.predict(val_model, val_set);

best_threshold_glm, f1_score_glm = find_best_threshold(val_pred_glm, val_labels)

(0.2, 0.6938775510204082)

In [223]:
evaluate_threshold(val_pred_glm, val_labels, 0.5) 

0.6341463414634146

#### Train Random Forest

In [ ]:
best_params, f1_score_rf = find_best_rf(train_set, val_set, names_ft, params_rf)

#### Train ensemble model

Get probabilities for RF

In [ ]:
val_pred_rf = get_rf_probas(train_set, val_set, names_ft, best_params)

Combine them

In [ ]:
val_pred = (val_pred_glm + val_pred_rf) ./ 2;
best_threshold, f1_score = find_best_threshold(val_pred, val_labels)

#### Get full model

In [ ]:
test_set = CSV.read("data/parsed/test_3260.csv");
size(test_set)

In [ ]:
test_model_glm = glm(val_form, data_set, Bernoulli(), LogitLink());
test_pred_glm = GLM.predict(test_model_glm, test_set);

In [ ]:
test_pred_rf = get_rf_probas(data_set, test_set, names_ft, best_params)

In [ ]:
test_pred = (test_pred_glm + test_pred_rf) ./ 2;

In [224]:
test_pred[test_pred .>= best_threshold] .= 1.0;
test_pred[test_pred .< best_threshold] .= 0.0;
test_pred = convert(Array{Int}, trunc.(test_pred));

#### Get prediction for ouvrage 3260

In [ ]:
pred_3260 = test_set;
pred_3260[!, :SURVERSE] = test_pred;

## Ouvrage 3350

In [196]:
data_3350 = CSV.read("data/parsed/ouvrage_3350.csv");
train_set, val_set = partitionTrainTest(data_3350);

#### GLM Train / Val

In [197]:
val_model = glm(val_form, train_set, Bernoulli(), LogitLink())
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_set);

find_best_threshold(val_pred, val_labels)

(0.2, 0.8125)

In [198]:
evaluate_threshold(val_pred, val_labels, 0.5) 

0.7037037037037037

#### GLM Submission

In [13]:
test_model = glm(val_form, data_3350, Bernoulli(), LogitLink());

In [113]:
test_3350 = CSV.read("data/parsed/test_3350.csv");
size(test_3350)

(70, 16)

In [15]:
test_pred = GLM.predict(test_model, test_3350);
threshold = 0.5;
test_pred[test_pred .>= threshold] .= 1.0;
test_pred[test_pred .< threshold] .= 0.0;
test_pred = convert(Array{Int}, trunc.(test_pred));

In [16]:
pred_3350 = test_3350;
pred_3350[!, :SURVERSE] = test_pred;

In [17]:
first(pred_3350[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-01,1,79.0
2,2019-05-02,0,26.0
3,2019-05-08,0,0.0
4,2019-05-09,1,89.0
5,2019-05-10,1,385.0


#### Decision Tree

In [114]:
dt_features = convert(Matrix{Float64}, data_3350[:, names_ft]);
dt_labels = convert(Array{Int64}, data_3350[!,:SURVERSE]);
dt_model = build_tree(dt_labels, dt_features);

dt_model = prune_tree(dt_model, 0.9)

dt_to_pred = convert(Matrix{Float64}, test_3350[:, names_ft]);
test_pred = apply_tree(dt_model, dt_to_pred);

pred_3350 = test_3350;
pred_3350[!, :SURVERSE] = test_pred;

In [115]:
first(pred_3350[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-01,0,79.0
2,2019-05-02,0,26.0
3,2019-05-08,0,0.0
4,2019-05-09,1,89.0
5,2019-05-10,1,385.0


## Ouvrage 4240

In [116]:
data_4240 = CSV.read("data/parsed/ouvrage_4240.csv");
train_set, val_set = partitionTrainTest(data_4240);

#### GLM Train / Val

In [19]:
val_model = glm(val_form, train_set, Bernoulli(), LogitLink())
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_set);

threshold = 0.5;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred));

r = roc(val_labels, val_pred);
f1score(r)

0.7619047619047619

#### GLM Submission

In [20]:
test_model = glm(val_form, data_4240, Bernoulli(), LogitLink());

In [117]:
test_4240 = CSV.read("data/parsed/test_4240.csv");
size(test_4240)

(49, 16)

In [22]:
test_pred = GLM.predict(test_model, test_4240);
threshold = 0.5;
test_pred[test_pred .>= threshold] .= 1.0;
test_pred[test_pred .< threshold] .= 0.0;
test_pred = convert(Array{Int}, trunc.(test_pred));

In [23]:
pred_4240 = test_4240;
pred_4240[!, :SURVERSE] = test_pred;

In [24]:
first(pred_4240[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-07,0,3.0
2,2019-05-09,0,89.0
3,2019-05-10,1,385.0
4,2019-05-15,0,2.0
5,2019-05-21,0,0.0


#### Decision Tree

In [118]:
dt_features = convert(Matrix{Float64}, data_4240[:, names_ft]);
dt_labels = convert(Array{Int64}, data_4240[!,:SURVERSE]);
dt_model = build_tree(dt_labels, dt_features);

dt_model = prune_tree(dt_model, 0.9)

dt_to_pred = convert(Matrix{Float64}, test_4240[:, names_ft]);
test_pred = apply_tree(dt_model, dt_to_pred);

pred_4240 = test_4240;
pred_4240[!, :SURVERSE] = test_pred;

In [119]:
first(pred_4240[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-07,0,3.0
2,2019-05-09,0,89.0
3,2019-05-10,1,385.0
4,2019-05-15,0,2.0
5,2019-05-21,0,0.0


## Ouvrage 4350

In [120]:
data_4350 = CSV.read("data/parsed/ouvrage_4350.csv");
train_set, val_set = partitionTrainTest(data_4350);

#### GLM Train / Val

In [26]:
val_model = glm(val_form, train_set, Bernoulli(), LogitLink())
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_set);

threshold = 0.5;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred));

r = roc(val_labels, val_pred);
f1score(r)

0.6666666666666666

#### GLM Submission

In [27]:
test_model = glm(val_form, data_4350, Bernoulli(), LogitLink());

In [121]:
test_4350 = CSV.read("data/parsed/test_4350.csv");
size(test_4350)

(65, 16)

In [29]:
test_pred = GLM.predict(test_model, test_4350);
threshold = 0.5;
test_pred[test_pred .>= threshold] .= 1.0;
test_pred[test_pred .< threshold] .= 0.0;
test_pred = convert(Array{Int}, trunc.(test_pred));

In [30]:
pred_4350 = test_4350;
pred_4350[!, :SURVERSE] = test_pred;

In [31]:
first(pred_4350[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-03,0,34.0
2,2019-05-04,0,2.0
3,2019-05-07,0,3.0
4,2019-05-08,0,0.0
5,2019-05-10,0,385.0


#### Decision Tree

In [122]:
dt_features = convert(Matrix{Float64}, data_4350[:, names_ft]);
dt_labels = convert(Array{Int64}, data_4350[!,:SURVERSE]);
dt_model = build_tree(dt_labels, dt_features);

dt_model = prune_tree(dt_model, 0.9)

dt_to_pred = convert(Matrix{Float64}, test_4350[:, names_ft]);
test_pred = apply_tree(dt_model, dt_to_pred);

pred_4350 = test_4350;
pred_4350[!, :SURVERSE] = test_pred;

In [123]:
first(pred_4350[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-03,0,34.0
2,2019-05-04,0,2.0
3,2019-05-07,0,3.0
4,2019-05-08,0,0.0
5,2019-05-10,1,385.0


## Ouvrage 4380

In [124]:
data_4380 = CSV.read("data/parsed/ouvrage_4380.csv");
train_set, val_set = partitionTrainTest(data_4380);

#### GLM Train / Val

In [33]:
val_model = glm(val_form, train_set, Bernoulli(), LogitLink())
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_set);

threshold = 0.5;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred));

r = roc(val_labels, val_pred);
f1score(r)

0.7027027027027027

#### GLM Submission

In [34]:
test_model = glm(val_form, data_4380, Bernoulli(), LogitLink());

In [125]:
test_4380 = CSV.read("data/parsed/test_4380.csv");
size(test_4380)

(54, 16)

In [36]:
test_pred = GLM.predict(test_model, test_4380);
threshold = 0.5;
test_pred[test_pred .>= threshold] .= 1.0;
test_pred[test_pred .< threshold] .= 0.0;
test_pred = convert(Array{Int}, trunc.(test_pred));

In [37]:
pred_4380 = test_4380;
pred_4380[!, :SURVERSE] = test_pred;

In [38]:
first(pred_4380[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-02,0,26.0
2,2019-05-04,0,2.0
3,2019-05-05,0,0.0
4,2019-05-06,0,0.0
5,2019-05-08,0,0.0


#### Decision Tree

In [126]:
dt_features = convert(Matrix{Float64}, data_4380[:, names_ft]);
dt_labels = convert(Array{Int64}, data_4380[!,:SURVERSE]);
dt_model = build_tree(dt_labels, dt_features);

dt_model = prune_tree(dt_model, 0.9)

dt_to_pred = convert(Matrix{Float64}, test_4380[:, names_ft]);
test_pred = apply_tree(dt_model, dt_to_pred);

pred_4380 = test_4380;
pred_4380[!, :SURVERSE] = test_pred;

In [127]:
first(pred_4380[!, [:DATE, :SURVERSE, :McTavish_sum]], 5)

,DATE,SURVERSE,McTavish_sum
,Date,Int64,Float64
1,2019-05-02,0,26.0
2,2019-05-04,0,2.0
3,2019-05-05,0,0.0
4,2019-05-06,0,0.0
5,2019-05-08,0,0.0


# Prédiction

In [129]:
test_set = CSV.read("data/test.csv");
test_set[!, :SURVERSE] = zeros(size(test_set, 1));
size(test_set)

(283, 3)

Pour chaque ligne de test_set

    - On check l'ID de l'ouvrage pour savoir quel prediction load
    - On va chercher la prediction à telle date pour cet ouvrage
    - On le met à la ligne courante

In [130]:
for i=1:size(test_set, 1)
    curr_ouvrage = test_set[i, 1];
    pred_to_use = nothing;
    if curr_ouvrage == "3260-01D"
        pred_to_use = pred_3260;
#         println("3260-01D");
    elseif curr_ouvrage == "3350-07D"
        pred_to_use = pred_3350;
#         println("3350-07D");
    elseif curr_ouvrage == "4240-01D"
        pred_to_use = pred_4240;
#         println("4240-01D");
    elseif curr_ouvrage == "4350-01D"
        pred_to_use = pred_4350;
#         println("4350-01D");
    elseif curr_ouvrage == "4380-01D"
#         println("4380-01D");
        pred_to_use = pred_4380;
    end
    
    curr_date = test_set[i, :DATE];
    pred_res = filter(row -> row.DATE == curr_date, pred_to_use);
    
    test_set[i, :SURVERSE] = pred_res[1, :SURVERSE];
end

In [131]:
test_set[!, :SURVERSE] = convert(Array{Int}, test_set[!, :SURVERSE]);
first(test_set, 10)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,3260-01D,2019-05-02,0
2,3260-01D,2019-05-09,0
3,3260-01D,2019-05-10,1
4,3260-01D,2019-05-15,0
5,3260-01D,2019-05-20,0
6,3260-01D,2019-05-23,0
7,3260-01D,2019-05-24,0
8,3260-01D,2019-05-26,0
9,3260-01D,2019-05-30,0


In [132]:
ID = test_set[:,:NO_OUVRAGE].*"_".*string.(test_set[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=test_set[:, :SURVERSE])
CSV.write("submissions/mc-submission-18.csv",sampleSubmission)

"submissions/mc-submission-18.csv"